In [1]:
# !pip3 install mediapipe opencv-python jupyterthemes

# REFERENCE
<img src=https://i.imgur.com/qpRACer.png />

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import firebase_admin
from firebase_admin import credentials, db

# <font color=red> INIT DATABASE </font>

In [2]:
cred = credentials.Certificate("keys/touri-65f07-firebase-adminsdk-wuv71-3751c21aa8.json")
firebase_admin.initialize_app(cred, {'databaseURL': 'https://touri-65f07-default-rtdb.firebaseio.com/'})

# <font color=red> INIT MEDIAPIPE </font>

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

## <font color=purple> HELPERS </font>

In [4]:
def get_image():
    ret, frame = cap.read()
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)
    image.flags.writeable = False

    return image

In [5]:
def draw_landmarks(results, image):
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    point_style = mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4)
    joint_style = mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4)
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, point_style, joint_style)

    return image

In [6]:
def get_joint_angles(restult, joint_1, joint_2, joint_3):
    # LOOP THROUGH HANDS
    for hand in restult.multi_hand_landmarks:
        # NP FOR COMPUTATIONAL EFFICIENCY
        j1 = np.array([hand.landmark[joint_1].x, hand.landmark[joint_1].y]) 
        j2 = np.array([hand.landmark[joint_2].x, hand.landmark[joint_2].y]) 
        j3 = np.array([hand.landmark[joint_3].x, hand.landmark[joint_3].y]) 

        j32_radians = np.arctan2(j3[1] - j2[1], j3[0] - j2[0])
        j12_radians = np.arctan2(j1[1] - j2[1], j1[0] - j2[0])
        angle = j32_radians - j12_radians
        angle = np.abs(angle*180.0/np.pi)
        if angle > 180: angle = 360 - angle

        return round(angle, 2)

# <font color=red> MAIN </font>

In [18]:
cap = cv2.VideoCapture(1)
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=1) as hands:
    last_wrist_angle = 180 
    last_wrist_yaw = 0 
    while cap.isOpened():
        # GET IMAGE
        image = get_image()
        # PROCESS IMAGE
        result = hands.process(image)
        # DRAW ON IMAGE
        image = draw_landmarks(result, image)

        if result.multi_hand_landmarks:
            # CALCULATE INDEX ANGLE - JOINTS 8,6,5
            current_wrist_angle = get_joint_angles(result, 20, 17, 0)
            # CALCULATE INDEX ANGLE - JOINTS 8,6,5
            current_yaw_angle = get_joint_angles(result, 5, 0, 17)

            # UPDATE DATABASE - WRIST
            if abs(last_wrist_angle - current_wrist_angle) > 1:
                db.reference("/hand").update({"wrist_angle": current_wrist_angle})
            last_wrist_angle = current_wrist_angle
            # UPDATE DATABASE - YAW
            if abs(last_wrist_yaw - current_yaw_angle) > 1:
                db.reference("/hand").update({"wrist_yaw": current_yaw_angle})
            last_wrist_yaw = current_yaw_angle


            # IMAGE TEXT
            angle_text = "Wrist: {}     Palm: {}".format(current_wrist_angle, current_yaw_angle)
            cv2.putText(image, angle_text, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        #SHOW IMAGE
        cv2.imshow('Hand Tracking', image)
        # EXIT
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1